In [1]:
"""
Created on Mar 05 2022
@author: José Antonio Espinosa Melchor
         Jerónimo Arenas García

Temporary routine for generation of CSV datasets
for demonstration purposes

It is a "Fake Data Mediator" for use with the first
version of the Interactive Topic Model Trainer
"""

import argparse
import json
import os
from pathlib import Path
from langdetect import detect
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains, concat_ws, col, udf
from pyspark.sql.types import StringType

In [2]:
p = "ITMTrainer/datasets/S2ART"
current_path = Path(os.getcwd())
path_dataset = current_path.joinpath(p)
path_config = path_dataset.joinpath("config.json")
path_csv = path_dataset.joinpath("CSV")
with open(path_config, 'r') as fin:
    datasetMeta = json.load(fin)

query = datasetMeta["query"]
print("query is: ", query)


query is:  SELECT id,lemmas FROM parquet.`/export/ml4ds/IntelComp/Datalake/SemanticScholar/20220201/papers.parquet` WHERE array_contains(fieldsOfStudy, 'Art')


In [3]:
# Read subtable from parquet file
S2papers = spark.sql(query.replace("lemmas", "title, paperAbstract"))

#Concatenate text fields to lemmatize
S2papers = (
    S2papers.withColumn("rawtext",concat_ws('. ', "title", "paperAbstract"))
    .drop("title")
    .drop("paperAbstract")
)

def my_detect(rawt):
    try:
        return detect(rawt)
    except:
        return "na"

udf_detect = udf(lambda x:my_detect(x), StringType() )
S2papers = S2papers.withColumn("language",udf_detect(col("rawtext"))).select("id","rawtext","language")
S2papers = (
    S2papers.filter(col("language")=="en")
    .drop("language")
)

#Lemmatize rawtext column
S2papers = S2papers.withColumnRenamed("rawtext", "lemmas")

22/03/06 12:54:02 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/03/06 12:54:02 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/03/06 12:54:04 WARN metastore.ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/03/06 12:54:04 WARN metastore.ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jarenas@192.168.148.225
22/03/06 12:54:04 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/03/06 12:54:04 WARN metastore.ObjectStore: Failed to get database parquet, returning NoSuchObjectException


In [4]:
#Save corpus
#Maybe it could be better to leave it distributed if some WP3 tools can benefit from it?
S2papers.repartition(25).write.option("header",True).csv(f"file://{path_csv.as_posix()}")